# Notebook to generate indicators module

In [107]:
from mintalib import core

from inspect import Signature, Parameter

from __project__ import PKGDIR

In [108]:
import importlib.util

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [109]:
def core_functions():
    return list(k for k, v in vars(core).items() if k.isupper() and callable(v))

core_functions()

['AVGPRICE',
 'TYPPRICE',
 'WCLPRICE',
 'MIDPRICE',
 'PRICE',
 'CROSSOVER',
 'CROSSUNDER',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'LOG',
 'EXP',
 'ROC',
 'DIFF',
 'MIN',
 'MAX',
 'SUM',
 'MAD',
 'STDEV',
 'SMA',
 'EMA',
 'RMA',
 'WMA',
 'HMA',
 'DEMA',
 'TEMA',
 'MA',
 'RSI',
 'PLUSDI',
 'MINUSDI',
 'ADX',
 'TRANGE',
 'ATR',
 'NATR',
 'LATR',
 'SAR',
 'CCI',
 'CMF',
 'MFI',
 'BOP',
 'BBANDS',
 'KELTNER',
 'KER',
 'KAMA',
 'MACD',
 'PPO',
 'SLOPE',
 'RVALUE',
 'FORECAST',
 'STOCH',
 'STREAK',
 'EVAL']

In [110]:
# Sample code for indicator

from mintalib.core import wrap_function
from mintalib.model import FuncIndicator

@wrap_function(core.SMA)
def SMA(period: int = 3):
    return FuncIndicator.wrap(core.SMA, params=locals())


In [113]:
PREAMBLE = '''
""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_function
from .model import FuncIndicator

nan = float('nan')


'''



def make_indicator(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = func.__name__.upper()

    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    def annotate(param):
        if param.annotation == "int":
            return param.replace(annotation=int)

        if param.annotation == "str":
            return param.replace(annotation=str)

        if type(param.default) in (int, float, bool):
            return param.replace(annotation=type(param.default))

        return param


    newparams = parameters[1:]
    newparams = [annotate(p) for p in newparams]

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_function({qname})\n"
    code += f"def {name}{newsig}:\n"
    code += f"    return FuncIndicator.wrap({qname}, params=locals())\n"


    return code


print(make_indicator(core.EMA))

@wrap_function(core.EMA)
def EMA(period: int, *, adjust: bool = False, item: str = None):
    return FuncIndicator.wrap(core.EMA, params=locals())



In [114]:
def test_indicator(func, name=None, verbose=False):
    code = make_indicator(func, name=name, preamble=True, verbose=verbose)
    print(code)
    return new_module("mintalib.functions", code)


test_indicator(core.EMA)


""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_function
from .model import FuncIndicator

nan = float('nan')


@wrap_function(core.EMA)
def EMA(period: int, *, adjust: bool = False, item: str = None):
    return FuncIndicator.wrap(core.EMA, params=locals())



<module 'mintalib.functions'>

In [115]:

def make_indicators(functions=core_functions()):
    buffer = PREAMBLE

    for name in functions:
        func = getattr(core, name)
        text = make_indicator(func)
        buffer += text + "\n\n"

    buffer += "__all__ = [name for name in dir() if name.isupper()]\n"

    return buffer


code = make_indicators()
print(code[:1024], "...")


""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_function
from .model import FuncIndicator

nan = float('nan')


@wrap_function(core.AVGPRICE)
def AVGPRICE():
    return FuncIndicator.wrap(core.AVGPRICE, params=locals())


@wrap_function(core.TYPPRICE)
def TYPPRICE():
    return FuncIndicator.wrap(core.TYPPRICE, params=locals())


@wrap_function(core.WCLPRICE)
def WCLPRICE():
    return FuncIndicator.wrap(core.WCLPRICE, params=locals())


@wrap_function(core.MIDPRICE)
def MIDPRICE():
    return FuncIndicator.wrap(core.MIDPRICE, params=locals())


@wrap_function(core.PRICE)
def PRICE(item: str = None):
    return FuncIndicator.wrap(core.PRICE, params=locals())


@wrap_function(core.CROSSOVER)
def CROSSOVER(level: float = 0.0, *, item: str = None):
    return FuncIndicator.wrap(core.CROSSOVER, params=locals())


@wrap_function(core.CROSSUNDER)
def CROSSUNDER(level: float = 0.0, *, item: str = None):
 

In [116]:
code = make_indicators()

module = new_module("mintalib.indicators", code)

dir(module)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'DEMA',
 'DIFF',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'FORECAST',
 'FuncIndicator',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LATR',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MFI',
 'MIDPRICE',
 'MIN',
 'MINUSDI',
 'NATR',
 'PLUSDI',
 'PPO',
 'PRICE',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TYPPRICE',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'nan',
 'wrap_function']

In [117]:
outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

Updating indicators.py ...


7241

In [118]:
from mintalib import indicators

In [119]:
help(indicators)

Help on module mintalib.indicators in mintalib:

NAME
    mintalib.indicators - Mintalib Indicators

FUNCTIONS
    ADX(period: int = 14)
        Average Directional Index
    
    ATR(period: int = 14)
        Average True Range
    
    AVGPRICE()
        Average Price
    
    BBANDS(period: int = 20, nbdev: float = 2.0)
        Bollinger Bands
    
    BOP(period: int = 20)
        Balance of Power
    
    CCI(period: int = 20)
        Commodity Channel Index
    
    CMF(period: int = 20)
        Chaikin Money Flow
    
    CROSSOVER(level: float = 0.0, *, item: str = None)
        Cross Over
    
    CROSSUNDER(level: float = 0.0, *, item: str = None)
        Cross Under
    
    DEMA(period: int, *, item: str = None)
        Double Exponential Moving Average
    
    DIFF(period: int = 1, *, item: str = None)
        Difference
    
    EMA(period: int, *, adjust: bool = False, item: str = None)
        Exponential Moving Average
        
        Args:
            series (series

In [120]:
indicators.SMA(20)

SMA(20)